In [1]:
%matplotlib inline
import math
import pandas as pd
from tsmom_model import *
import numpy as np
from datetime import datetime
import PPCA_MR as p
import statsmodels.api as sm

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=2)
pylab.rcParams['figure.figsize'] = (24, 18)

# Hand cleansed
Hand cleaned to removed duplicates

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:161: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:163: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


# Amihud methodology

Amihud (2002) developed the illiquidity measure expressed by:

$$ Amihud =\frac{1}{N} \sum \frac{|r_t|}{\text{dvol}_t} $$

where $r_t$ is the return on day $t$ and $Volume_t$ is the dollar volume on day $t$. 

The benefit of taking the daily return is to weight the move by change of price and the volume.


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

# Academic papers
Based on Illin, A., Raiko, T. 2010 paper called "Practical Approaches to Principal Component Analysis in the Presence of Missing Values"

Tipping and Bishop (1999) introducted probablistic formulation of PCA (PPCA)


Use a flavor of Probabilistic PCA that is robust to missing data (see Ilin and Raiko 2010). Rather than the vanilla one-shot SVD, PPCA uses an iterative EM procedure/fixed point algorithm. From an initial guess, it’ll alternatively interpolate missing data and update the components until convergence.

On a side note, PPCA interpolates using information from all series, making it a multivariate interpolator. Be careful though, fitting the model to too many components will likely lead to overfitting problems in the interpolated data.

Also note that I am using the standardization that is mentioned in Korajecyk and Sadka (2008) of expanding zscores


In [120]:
sector_resid={}
sector_rsq={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_data={}
    rsq=pd.Series()
    for mkt in d_map[sect]:
        ind2=amihud[mkt].dropna().index
        ind=amihud[mkt].shift().loc[ind2].dropna().index
        res=sm.OLS(amihud[mkt].shift().loc[ind],amihud[mkt].loc[ind]).fit()
        sector_data[mkt]=res.resid
        rsq[mkt]=res.rsquared
    sector_resid[sect]=pd.DataFrame(sector_data)
    sector_rsq[sect]=rsq.mean()


# FHT method

In [128]:
FHT=calculate_FHT(cleansed)

tsmom_model.py:270: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
tsmom_model.py:271: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
tsmom_model.py:272: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
tsmom_model.py:272: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')


In [129]:
sector_resid_FHT={}
sector_rsq_FHT={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_data={}
    rsq=pd.Series()
    for mkt in d_map[sect]:
        ind2=FHT[mkt].dropna().index
        ind=FHT[mkt].shift().loc[ind2].dropna().index
        res=sm.OLS(FHT[mkt].shift().loc[ind],FHT[mkt].loc[ind]).fit()
        sector_data[mkt]=res.resid
        rsq[mkt]=res.rsquared
    sector_resid_FHT[sect]=pd.DataFrame(sector_data)
    sector_rsq_FHT[sect]=rsq.mean()

Commodities     0.607543
Currencies      0.701559
Equities        0.726504
Fixed Income    0.515586
dtype: float64

# Presentation
First rsquares of residuals

In [144]:
results_rsq=pd.DataFrame()
results_rsq['Amihud']=pd.Series(sector_rsq)
results_rsq['FHT']=pd.Series(sector_rsq_FHT)
print results_rsq.round(2).to_latex()

\begin{tabular}{lrr}
\toprule
{} &  Amihud &   FHT \\
\midrule
Commodities  &    0.72 &  0.61 \\
Currencies   &    0.78 &  0.70 \\
Equities     &    0.75 &  0.73 \\
Fixed Income &    0.64 &  0.52 \\
\bottomrule
\end{tabular}

